In [116]:
import requests
import json
from datetime import datetime
import math

In [117]:
base_url = "https://ergast.com/api/f1/"

In [118]:
def getPrevResults(year_prev, round_prev):
    all_prev_results = []
    res_url_prev = base_url + str(year_prev) + '/' + str(round_prev) + '/results.json'
    data_prev = json.loads(requests.get(res_url_prev).text)
    rt_prev = data_prev['MRData']['RaceTable']
    for race in rt_prev['Races']:
        results_prev = race['Results']
        for result in results_prev:
            driver_id = result['Driver']['driverId']
            pos_prev = result['position']
            if round_prev != getLastRoundOfYear(year_prev):
                pts = result['points']
            else:
                pts = "0"
            all_prev_results.append({"driver": driver_id,
                                     "position": pos_prev,
                                     "points": pts})
    return all_prev_results


def getLastRoundOfYear(year):
    season_url = base_url + str(year) + '.json'
    data_json = json.loads(requests.get(season_url).text)
    race_table = data_json['MRData']['RaceTable']
    return int(race_table['Races'][-1]['round'])


def getNumberOfRoundsInYear(year):
    season_url = base_url + str(year) + '.json'
    data_json = json.loads(requests.get(season_url).text)
    race_table = data_json['MRData']['RaceTable']
    return len(race_table['Races'])

In [119]:
def retrieveResultObject(year, round_id):
    res_url = base_url + str(year) + '/' + str(round_id) + '/results.json'
    data_json = json.loads(requests.get(res_url).text)
    race_table = data_json['MRData']['RaceTable']
    return race_table['Races'][0]


def retrieveDriverObject(year, round_id):
    driver_url = base_url + str(year) + '/' + str(round_id) + '/drivers.json'
    data_json = json.loads(requests.get(driver_url).text)
    driver_table = data_json['MRData']['DriverTable']
    return driver_table


def retrieveConstructorObject(year, round_id):
    team_url = base_url + str(year) + '/' + str(round_id) + '/constructors.json'
    data_json = json.loads(requests.get(team_url).text)
    team_table = data_json['MRData']['ConstructorTable']
    return team_table


def getRaceDate(year, round_id):
    data_json = json.loads(requests.get(base_url + str(year) + '.json').text)
    round_index = round_id - 1
    race_list = data_json['MRData']['RaceTable']['Races']
    race_obj = race_list[round_index]
    return race_obj['date']


def calculateAge(birthday, race_date):
    rd = datetime.strptime(race_date, "%Y-%m-%d")
    bd = datetime.strptime(birthday, "%Y-%m-%d")
    return math.floor(abs(rd - bd).days / 365)

In [120]:
def saveRaceInfo(race_obj, date):
    year = race_obj['season']
    circuit_id = race_obj['Circuit']['circuitId']
    circuit_name = race_obj['Circuit']['circuitName']
    race_round = race_obj['round']
    race_name = race_obj['raceName']
    country = race_obj['Circuit']['Location']['country']
    city = race_obj['Circuit']['Location']['locality']
    fn = './Ergast_Data_Circuit/' + year + '_' + circuit_id + '.json'
    new_obj = {
        "round": race_round, "date": date, "race_name": race_name,
        "circuit_name": circuit_name, "country": country, "city": city
    }
    with open(fn, 'w') as f:
        json.dump(new_obj, f)


def saveDriverInfo(driver_table, date, race_obj, prev_results):
    year = race_obj['season']
    circuit_id = race_obj['Circuit']['circuitId']
    results = race_obj['Results']
    new_objs = {}
    for driver in driver_table['Drivers']:
        driver_id = driver['driverId']
        driver_nr = driver['permanentNumber']
        driver_code = driver['code']
        first_name = driver['givenName']
        last_name = driver['familyName']
        birthday = driver['dateOfBirth']
        nationality = driver['nationality']
        team = [x['Constructor']['constructorId'] for x in results
                if x['Driver']['driverId'] == driver_id][0]
        age = calculateAge(birthday, date)
        start_pos = [x['grid'] for x in results
                     if x['Driver']['driverId'] == driver_id][0]
        try:
            prev_res = [x['position'] for x in prev_results
                        if x['driver'] == driver_id][0]
            champ_pts = [x['points'] for x in prev_results
                         if x['driver'] == driver_id][0]
        except IndexError:
            prev_res, champ_pts = None, None
        new_objs[driver_id] = {
            "number": driver_nr, "code": driver_code,
            "first_name": first_name, "last_name": last_name,
            "nationality": nationality, "team": team, "age": age,
            "start_position": start_pos, "previous_result": prev_res,
            "points": champ_pts
        }
    fn = './Ergast_Data_Drivers/' + year + '_' + circuit_id + '.json'
    with open(fn, 'w') as f:
        json.dump(new_objs, f)
    return new_objs


def saveConstructorInfo(const_table, race_obj, driver_info):
    year = const_table['season']
    circuit_id = race_obj['Circuit']['circuitId']
    new_objs = {}
    for constructor in const_table['Constructors']:
        const_id = constructor['constructorId']
        const_name = constructor['name']
        nationality = constructor['nationality']
        drivers = [x for x in driver_info if driver_info[x]['team'] == const_id]
        driver_1, driver_2 = drivers
        new_objs[const_id] = {
            "name": const_name,
            "nationality": nationality,
            "driver_1": driver_1,
            "driver_2": driver_2
        }
    fn = './Ergast_Data_Constructors/' + year + '_' +\
        circuit_id + '.json'
    with open(fn, 'w') as f:
        json.dump(new_objs, f)

In [121]:
test = {"fgd": {"gfgk": 1}, "fdfd": {"gfgk": 1}}
fd = [x for x in test if test[x]['gfgk'] == 1]
fd

['fgd', 'fdfd']

In [ ]:
def getDataForRound(year, round_id):
    if round_id > 1:
        prev_results = getPrevResults(year, round_id-1)
    else:
        prev_results = getPrevResults(year-1, getLastRoundOfYear(year-1))
    date = getRaceDate(year, round_id)
    race_obj = retrieveResultObject(year, round_id)
    saveRaceInfo(race_obj, date)
    driver_table = retrieveDriverObject(year, round_id)
    const_table = retrieveConstructorObject(year, round_id)
    driver_info = saveDriverInfo(driver_table, date, race_obj, prev_results)
    saveConstructorInfo(const_table, race_obj, driver_info)


def getDataForYear(year):
    nr_rounds = getNumberOfRoundsInYear(year)
    for round_id in range(1, nr_rounds + 1):
        getDataForRound(year, round_id)


for year in range(2018, 2024):
    getDataForYear(year)